# Template - Strategy - Backtesting 

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import candlestick.price_action as pa
from backtesting.backtesting import Backtest, Strategy
# from backtesting._plotting import set_bokeh_output
# set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

/home/zuongthao/PycharmProjects/algo-stock/venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [5]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-07 14:15:00,1343.3,1344.4,1342.4,1344.2,5075
2025-02-07 14:20:00,1344.2,1344.7,1343.6,1344.0,4865
2025-02-07 14:25:00,1344.1,1344.2,1342.8,1343.3,5013


In [6]:
def has_bullish_pattern(model):
    if "bullish" in model or "rising" in model:
        return True
    return False


def has_bearish_pattern(model):
    if "bearish" in model or "falling" in model:
        return True
    return False

In [7]:
def prepare_data(data):
    data['RSI'] = ta.rsi(data['Close'], length=14)
    data['EMA_20'] = ta.ema(data['Close'], length=20)
    data['EMA_250'] = ta.ema(data['Close'], length=250)
    data['max_5'] = data['High'].rolling(5).max()
    data['min_5'] = data['Low'].rolling(5).min()
    data = pa.pattern_modeling(data, 'reversal')
    data['signal'] = data.apply(lambda row: 'long' if row['RSI'] < 41 and has_bullish_pattern(row['model']) and row['EMA_20'] > row['EMA_250'] else '', axis=1)
    data['close_signal'] = data.apply(lambda row: 'close' if (row['RSI'] > 65 and has_bearish_pattern(row['model'])) or row['Close'] < row['max_5'] - 4 else '', axis=1)
    return data

In [16]:
class MainStrategy(Strategy):
    def init(self):
        super().init()

    def next(self):
        super().next()
        if self.position:
            close_signal = self.data.close_signal[-1]
            if close_signal == 'close':
                self.position.close()
                
        else:
            signal = self.data.signal[-1]
            if signal == 'long':
                low = self.data.Low[-1]
                low_1 = self.data.Low[-2]
                sl = min(low, low_1) - 0.3
                self.buy(size=1, sl=sl)

In [9]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [10]:
prepared_data[prepared_data.model == 'bullish_piercing']

,Open,High,Low,Close,Volume,RSI,EMA_20,EMA_250,max_5,min_5,...,max_OC,upper_wick,tail,oc_dif,body,color,candlestick,model,signal,close_signal
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-10 09:30:00,933.0,933.4,932.7,933.3,1555,82.438216,926.526272,913.165778,934.3,932.7,...,933.3,0.1,0.3,0.3,0.3,white,,bullish_piercing,,
2020-11-10 11:05:00,928.1,930.0,927.5,929.1,2293,53.619375,928.936352,915.523984,930.0,927.2,...,929.1,0.9,0.6,1.0,1.0,white,,bullish_piercing,,
2020-11-10 11:25:00,927.6,929.4,927.6,929.4,1275,53.833890,928.901989,915.942897,930.0,927.5,...,929.4,0.0,0.0,1.8,1.8,white,marubozu,bullish_piercing,,
2020-11-10 13:15:00,929.0,929.3,928.1,929.3,1497,52.938438,929.056834,916.466626,930.2,928.1,...,929.3,0.0,0.9,0.3,0.3,white,hammer,bullish_piercing,,
2020-11-11 09:20:00,921.1,921.9,921.0,921.5,2208,30.562889,925.462058,918.062706,922.6,921.0,...,921.5,0.4,0.1,0.4,0.4,white,,bullish_piercing,long,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-04 14:10:00,1327.5,1329.1,1327.4,1328.4,2996,57.704899,1327.077569,1326.127770,1329.8,1326.9,...,1328.4,0.7,0.1,0.9,0.9,white,,bullish_piercing,,
2025-02-05 10:35:00,1335.4,1336.0,1335.4,1335.8,1096,63.711565,1334.295715,1327.508571,1337.8,1335.3,...,1335.8,0.2,0.0,0.4,0.4,white,,bullish_piercing,,
2025-02-05 11:25:00,1331.8,1332.1,1331.6,1332.0,1162,40.981359,1333.531723,1327.965157,1333.4,1331.0,...,1332.0,0.1,0.2,0.2,0.2,white,,bullish_piercing,long,


In [11]:
prepared_data[prepared_data.model != ''][['RSI', 'EMA_20', 'EMA_250', 'model', 'signal', 'close_signal']].tail(20)

,RSI,EMA_20,EMA_250,model,signal,close_signal
Date,,,,,,
2025-02-06 14:00:00,54.198885,1340.350230,1332.397623,bearish_meeting_line,,
2025-02-06 14:05:00,66.239680,1340.650208,1332.486089,bullish_tasuki_line,,
2025-02-06 14:10:00,54.393744,1340.731141,1332.557913,bearish_dark_cloud_cover,,
2025-02-07 09:10:00,37.510732,1339.536597,1332.949400,bearish_dark_cloud_cover,,
2025-02-07 09:20:00,38.699969,1338.864199,1332.994685,bullish_piercing,long,
2025-02-07 09:40:00,44.718905,1338.519117,1333.146072,bearish_tasuki_line,,
2025-02-07 09:55:00,45.689539,1338.121847,1333.236713,bullish_engulfing,,
2025-02-07 10:10:00,53.128211,1338.350691,1333.373393,bullish_engulfing,,
2025-02-07 10:15:00,50.526877,1338.383958,1333.415836,bearish_tweezers_top,,


In [17]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [19]:
stats

Start                     2020-11-06 14:10:00
End                       2025-02-07 14:45:00
Duration                   1554 days 00:35:00
Exposure Time [%]                    7.178851
Equity Final [$]                   10206.4368
Equity Peak [$]                   10216.65947
Return [%]                           2.064368
Buy & Hold Return [%]               47.453348
Return (Ann.) [%]                    0.486959
Volatility (Ann.) [%]                0.516021
Sharpe Ratio                          0.94368
Sortino Ratio                        2.180392
Calmar Ratio                         0.527984
Max. Drawdown [%]                   -0.922298
Avg. Drawdown [%]                   -0.051338
Max. Drawdown Duration      582 days 20:50:00
Avg. Drawdown Duration       21 days 01:02:00
# Trades                                  346
Win Rate [%]                        29.479769
Best Trade [%]                       3.872935
Worst Trade [%]                     -0.588698
Avg. Trade [%]                    

In [20]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,1,113,114,921.67642,920.7,-0.97642,-0.001059,2020-11-11 09:25:00,2020-11-11 09:30:00,None,0 days 00:05:00
1,1,122,126,921.67642,919.8,-1.87642,-0.002036,2020-11-11 10:10:00,2020-11-11 10:30:00,None,0 days 00:20:00
2,1,128,137,921.27630,919.0,-2.27630,-0.002471,2020-11-11 10:40:00,2020-11-11 11:25:00,None,0 days 00:45:00
3,1,143,149,919.27570,917.6,-1.67570,-0.001823,2020-11-11 13:20:00,2020-11-11 13:50:00,None,0 days 00:30:00
4,1,282,285,934.68032,932.2,-2.48032,-0.002654,2020-11-16 10:55:00,2020-11-16 11:10:00,None,0 days 00:15:00
...,...,...,...,...,...,...,...,...,...,...,...
341,1,53558,53558,1320.29597,1319.0,-1.29597,-0.000982,2025-01-21 13:40:00,2025-01-21 13:40:00,None,0 days 00:00:00
342,1,53729,53731,1325.99768,1323.4,-2.59768,-0.001959,2025-02-03 09:30:00,2025-02-03 09:40:00,None,0 days 00:10:00
343,1,53855,53862,1332.39960,1330.7,-1.69960,-0.001276,2025-02-05 11:30:00,2025-02-05 13:30:00,None,0 days 02:00:00
344,1,53864,53884,1331.49933,1341.7,10.20067,0.007661,2025-02-05 13:40:00,2025-02-06 09:40:00,None,0 days 20:00:00


In [15]:
# bt.plot()